# 5. Model

## 5.0 Read Data

In [1]:
# check xgboost version
import xgboost
print(xgboost.__version__)
# evaluate an xgboost regression model on the housing dataset
from numpy import absolute
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, StratifiedKFold
from xgboost import XGBRegressor
import numpy as np

1.4.2


In [2]:
data = pd.read_csv('xtrain.csv',index_col=0)
target = pd.read_csv('train.csv')

In [3]:
# data.dropna(axis= 0, inplace=True)
# data.fillna(0, inplace=True)
target = data['SalePrice']
data.drop('SalePrice',inplace=True, axis=1)
# data.drop('LotFrontage', inplace=True, axis=1)
# data.replace(np.inf, 0, inplace=True)

In [4]:
target = np.log(target)

In [5]:
pd.set_option('display.max_columns', None)
data.head()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.12250,6.752270,0.413559,0.0,7.444249,0.666667,0.5,0.375,0.333333,0.000000,0.002488,0.386460,0.50,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.090909,0.50,4.174387,9.041922,0.125089,0.0,0.064212,0.333333,0.140098,0.0,0.500000
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [6]:
data.tail()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
1455,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,6.859615,0.336077,0.0,7.406711,0.666667,0.5,0.375,0.333333,0.333333,0.003980,0.324401,0.50,0.000000,0.073126,0.000000,0.0,0.0,0.0,0.636364,0.25,4.127134,8.976768,0.000000,0.000000,0.407962,0.000000,0.155974,0.0,0.416667
1456,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,

## 5.1 Train Test Split

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(data, target, test_size=0.3)

## 5.2 Linear Regression

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
reg = LinearRegression()

In [11]:
reg.fit(xtrain,ytrain)

LinearRegression()

In [ ]:
# define model
model = LogisticRegression()
# define search space
space = dict()
# define search
search = GridSearchCV(model, space)

In [ ]:
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [32]:

print('\nStandard Deviation is:', stdev(lst_accu_stratified))

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [12]:
reg.score(xtest, ytest)

-2.129841971677429e+22

In [14]:
y_pred = reg.predict(xtest)

In [23]:
from sklearn.metrics import mean_squared_error
errors = mean_squared_error(ytest, y_pred)


In [38]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=300, learning_rate=0.1)
regr.fit(xtrain,ytrain)

AdaBoostRegressor(learning_rate=0.1, n_estimators=300, random_state=0)

In [39]:
ypred = regr.predict(xtest)

In [40]:
from sklearn.metrics import mean_squared_error
errors = mean_squared_error(ytest, ypred)

In [41]:
errors

0.038081244372594296

In [ ]:
loss